# Crime In The US
Team 7: Lorenzo Gonzalez, Daisy Cheng, Ricardo Lira, Briana Ortega Vazquez

Here we start by reading in our clean datasets in which we renamed some colums and filled in some missing data values.
We display the first five rows of the datasets to make sure they are reading in correctly

In [ ]:
import pandas as pd
import numpy as np
import numpy
import matplotlib.pyplot as plt
import seaborn as sb
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from scipy.stats import chi2_contingency
import seaborn as sns
%matplotlib inline

USCrime_df = pd.read_csv("crimeInUS.csv")
StateCrime_df = pd.read_csv("crime_state.csv")

#Ricardo
USCrimeR_df = pd.read_csv("crimeInUS(Ricardo).csv")
USCrimeR_df = USCrimeR_df.replace(',','', regex=True)
StateCrime_df = pd.read_csv("crime_state(Ricardo).csv")

#Daisy
USCrimeD_df = pd.read_csv("crimeInUS-Daisy.csv")
USCrimeD_df = USCrimeD_df.replace(',','', regex=True)
USCrimeD_df = USCrimeD_df.astype(float)

display(USCrime_df.head(5))
display(StateCrime_df.head(5))
display(USCrimeR_df.head(5))
display(USCrimeD_df.head(5))

# Linear Regression (Multiple Regression)
Training our data to predict the rate of property crime and violent crime.
We train 80% and test 20% of our dataset.
We are using Burglary Rate, Larcent-theft Rate and Motor Vechile Theft Rate as factors for Property Crime.
We are using Murder/Manslaughter Rate, Robbery Rate and Aggravated Assault Rate as factors for Violent Crime Rate.

In [ ]:
USCrime_df = USCrime_df.replace(",", "", regex = True)

USCrime_train = USCrime_df.loc[0:16].copy()
USCrime_test = USCrime_df.loc[17:19].copy()

X_train = USCrime_train[["BurglaryRate", "Larceny-theftRate", "MotorVehicleTheftRate"]]
X_test = USCrime_test[["BurglaryRate", "Larceny-theftRate", "MotorVehicleTheftRate"]]
y_train = USCrime_train["PropertyCrimeRate"]


propertyCrimeModel = LinearRegression()
propertyCrimeModel.fit(X=X_train, y=y_train)
display(propertyCrimeModel.predict(X=X_test))

coef = propertyCrimeModel.coef_
intercept = propertyCrimeModel.intercept_

y = coef[0]*918.8 + coef[1]*1745 + coef[2]*505.7 + intercept

print("slope:", coef)
print("intercept:", intercept)
print("\nProperty Crime Rate using a high burglary and Motor Vehicle theft rate and a low larceny-theft rate:", y)

y = coef[0]*468.9 + coef[1]*2891.80 + coef[2]*236.9 + intercept

print("\nProperty Crime Rate using a low burglary and Motor Vehicle theft rate and a high larceny-theft rate:", y)

In [ ]:
X_train = USCrime_train[["MurderOrMSRate", "RobberyRate", "AggravatedAssaultRate"]]
X_test = USCrime_test[["MurderOrMSRate", "RobberyRate", "AggravatedAssaultRate"]]
y_train = USCrime_train["ViolentCrimeRate"]


violentCrimeModel = LinearRegression()
violentCrimeModel.fit(X=X_train, y=y_train)
display(violentCrimeModel.predict(X=X_test))

coef = violentCrimeModel.coef_
intercept = violentCrimeModel.intercept_

y = coef[0]*6.8 + coef[1]*102.8 + coef[2]*382.1 + intercept

print("slope:", coef)
print("intercept:", intercept)
print("\nViolent Crime Rate using a high murder/manslaughter and robbery rate and a low aggravated assault rate:", y)

y = coef[0]*5.3 + coef[1]*186.2 + coef[2]*248.5 + intercept

print("\nViolent Crime Rate using a low murder/manslaughter and robbery rate and a high aggravated assault rate:", y)

In [ ]:
USCrime_df.plot.bar(x = "Year", y = "ViolentCrimeRate", title = "Violent Crime Rate Per Year")
USCrime_df = USCrime_df.astype(float, errors = 'raise')
USCrime_df["Year"] = USCrime_df["Year"].astype(int, errors = 'raise')
USCrime_df.plot.bar(x = "Year", y = "PropertyCrimeRate",title = "Property Crime Rate Per Year")

By looking at the two graphs above you can see both Violent Crime and Property Crime rates have decreased since 1997. Although Property crime rates were slowly decreasing in 2006, Violent Crime rates went slightly up creating a very shallow bell curve in the graph.

In [ ]:
cor = pd.crosstab( USCrime_df["PropertyCrimeRate"], USCrime_df["ViolentCrimeRate"])
column_1 = USCrime_df["PropertyCrimeRate"]
column_2 = USCrime_df["ViolentCrimeRate"]
corr = column_1.corr(column_2, method = "spearman")
print('Spearman correlation: %.3f' % corr)


Since spearman's correlation is 0.962 which is close to 1 that means there is a strong correlation between Violent Crime Rates and Property Crime Rates.

# Correlation between Population and Arrests: A New Hypothesis

Below we will attempt to see if there is any correlation between Population and the amount of Arrests made. I hypothesize that with an increase in population, we will see a positive correlation with Arrests.

In [ ]:
USCrimeR_df = USCrimeR_df.astype(int)
USCrimeR_df_dropYear = USCrimeR_df.drop(["Year"], axis=1)
USCrimeR_df_dropYear.plot(title="Violent Crime by Population in the United States", y="ViolentCrime", x="Population")

USCrimeR_df_dropPop = USCrimeR_df.drop(["Population"], axis=1)
USCrimeR_df_dropPop.plot(title="Violent Crime by Year in the United States", y="ViolentCrime", x="Year")

Here we have a line graph with the Population on the X-axis and the amount of Violent Crime arrests made on the Y-axis followed by a near exact graph below it with the Year on the X-axis and the amount of Violent Crime arrests made on the Y-axis. This is because the Population is recorded on a given Year and shows that Population increases over time and by my hypothesis so should crime, yet so far it seems that Violent Crime is on a downward trend with some anamolies.

In [ ]:
USCrimeR_df_dropBoth = USCrimeR_df.drop(["Population", "Year"], axis=1)
USCrimeR_df_dropBoth["Total"] = USCrimeR_df_dropBoth.sum(axis=1)
USCrimeR_df["Total"] = USCrimeR_df_dropBoth["Total"]
USCrimeR_df.plot.scatter(title="Total Arrests in the United States", y="Total", x="Population")

column_1 = USCrimeR_df["Population"]
column_2 = USCrimeR_df["Total"]
corr = column_1.corr(column_2, method='spearman')
print('Spearman correlation: %.3f' % corr)

Here, we have what appears to be definitive proof that there is a strong negative correlation (-0.962) between Population and Crime/Number of Arrests, meaning that as Population increase, number of Arrests decreases. This would imply that my hypothesis was incorrect. However...

# Part of a Larger Story: Correlation between State Population and Arrests

Country-wide correlation between Arrests and Population have been on a downward trend, but we always hear that the higher the population, the more crime there is to be expected. Perhaps we can glean more insight by looking deeper into the issue. Below we will look at correlation between State Population and Arrests.

In [ ]:
StateCrime_df = StateCrime_df.replace(',','', regex=True)
States_df = StateCrime_df
States_df["State"] = States_df["State"].astype('|S80')
States_df["State"] = States_df["State"].astype(str)
States_df = States_df.replace("b'",'', regex=True)
States_df = States_df.replace("'",'', regex=True)
StateCrime_df = StateCrime_df.drop("State", axis=1)
StateCrime_df = StateCrime_df.astype(int)
States_df["Total"] = States_df['Total'].astype(int)
States_df["Population"] = States_df['Population'].astype(int)

x = StateCrime_df["Population"]
y = StateCrime_df["Total"]

mymodel = numpy.poly1d(numpy.polyfit(x, y, 3))

myline = numpy.linspace(1, 40000000, 40000000)

plt.scatter(x, y)
plt.plot(myline, mymodel(myline))
plt.show()

# StateCrime_train = StateCrime_df.loc[2:42].copy()
# StateCrime_test = StateCrime_df.loc[43:52].copy()
# StateCrime_train.plot.scatter(title="Total Arrests per State", x="Population", y="Total")

# X_train = StateCrime_df[["Population"]]
# X_test = StateCrime_df[["Population"]]
# y_train = StateCrime_df["Total"]

# model = LinearRegression()
# model.fit(X=X_train, y=y_train)
# model.predict(X=X_test)
# print(model.predict(X=X_test))
# X_new = pd.DataFrame()
# # create a sequence of 200 evenly spaced numbers from 10 to 41
# X_new["Population"] = np.linspace(0, 40000000, num=200)

# # create a Series out of the predicted values
# # (trailing underscore indicates fitted values)
# y_new_ = pd.Series(
#     model.predict(X_new), # y values in Series.plot.line()
#     index=X_new["Population"]    # x values in Series.plot.line()
# )

# # plot the data, then the model
# StateCrime_train.plot.scatter(title="Total Arrests per State", x="Population", y="Total")
# y_new_.plot.line()
# print(y_new_.plot.line())

Above is a Polynomial Regression of Population (x-axis) and Arrests (y-axis) and it shows that there is actually an upward trend between arrests and population, implying that the original hypothesis is actually correct. More details on this graph below:

In [ ]:
plt.rcParams['figure.figsize'] = (25, 7)
ax = sns.barplot(x = States_df['State'], y = States_df['Total'], palette = 'dark')
ax.set_xlabel(xlabel = 'State', fontsize = 10)
ax.set_ylabel(ylabel = 'Total Arrests', fontsize = 10)
ax.set_title(label = 'Arrests per State (2016)', fontsize = 20)
plt.xticks(rotation = 90)
plt.show()

plt.rcParams['figure.figsize'] = (25, 7)
ax = sns.barplot(x = States_df['State'], y = States_df['Population'], palette = 'dark')
ax.set_xlabel(xlabel = 'State', fontsize = 10)
ax.set_ylabel(ylabel = 'Population', fontsize = 10)
ax.set_title(label = 'Population per State (2016)', fontsize = 20)
plt.xticks(rotation = 90)
plt.show()

Above are 2 bar graphs to help visualize what was not shown in the Polynomial Regression. The first is the amount of Arrests made in each State. The second is the population size in each State. As we can see, the two graphs are almost identical, but careful observation shows us that these are indeed different graphs, but the ratio between crime and population is near 1:1.

In [ ]:
cor = pd.crosstab( StateCrime_df["Total"], StateCrime_df["Population"])
column_1 = StateCrime_df["Total"]
column_2 = StateCrime_df["Population"]
corr = column_1.corr(column_2, method="spearman")
print('Spearman correlation: %.3f' % corr)

Lastly, the Spearman correlation shows us a strong positive correlation of 0.952.But how can there be a negative trend across the US, but a strong positive trend on the state level?

There could be many explanations, but none of which are revealed by the data. For instance, maybe as a country we are looking at a downward trend in crime in general, but population does play a role in crime. Or perhaps fewer arrests are being made nationwide. Currently, we have no way to know for sure.

## Correlation between Aggravated Assault and Burglary Rate

In [ ]:
plt.rcParams['figure.figsize'] = (6, 4)
AA_Bur_year = USCrimeD_df[["Year", "AggravatedAssault", "BurglaryRate"]]
USCrimeD_df.plot.bar(x = "Year", y = "AggravatedAssault")
USCrimeD_df.plot.bar(x = "Year", y = "BurglaryRate")

By looking at the graph you can see that aggravated assault cases decreased from 1997 - 1999 then were a bit up and down from 1999 - 2013. Aggravated assault then started increasing 2014 onwards. Burglary rate also decreased from 1997 - 1999. However, burglary held pretty steady from 1999 - 2012. During that time you can see that burglary increases and decreases very slightly each year following the same trend as aggravated assault. The difference is that burglary rate start decreasing after 2012.

In [ ]:
AA_Bur = USCrimeD_df[["AggravatedAssault", "BurglaryRate"]]
corr1 = AA_Bur.corr(method = 'spearman')
print('Spearman\'s Correlation')
corr1

Since Spearman's correlation is 0.873684 which is close to 1 that means there is a pretty strong correlation between Aggravated Assault and Burglary Rate.


# Growing Transportation and Transportation Related Crime

With a growing population, there is a growing amount of new drivers, and with an influx of new drivers, there is a growing demand for cars. If you have ever been to urban cities such as San Francisco, then you know that there is constantly break in and vehicle robberies. Citizens have even gone as far as to put signs on their car windows saying "Please do not take my car!" We want to see if in increase in population does indeed lead to an increase in motor vehicle thefts.

In [ ]:
USCrime_df.plot(title = "Population by Year", y = "Population",x = "Year" )

In [ ]:
USCrime_df.plot.scatter(title="Violent Crime by Population", y="MotorVehicleTheft", x="Population")

In [ ]:
column_1 = USCrime_df["Population"]
column_2 = USCrime_df["MotorVehicleTheft"]
corr = column_1.corr(column_2, method = "spearman")
print('Spearman correlation: %.3f' % corr)

Although we see a constant rise in population year to year, we cannot say the same for motor vehicle thefts. In the dot graph, we see that there is a rise in motor vehicle thefts, it quicly goes down, but we do see it go up a bit on the end. Because of this the amounts of motor vehicle thefts per year is inconsistant and unpredictable. When we do the test for Spearman correlation, we get -0.859 meaning there is a very heavy negative correlation between motor vehicle thefts and Population. This means that the exact opposite of our conclusion is happening. As more people popluate the country, according to the Spearman test of correlation.